In [59]:
import numpy as np
import scipy
import utils
from utils import HouseHolder, QR, SVD

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Problem 1: SVD by Two-Phase Approach

## Phase-I: Golub-Kahan Bidiagonalization

In [60]:
%precision 8

'%.8f'

In [61]:
# A = np.array([[0, 0, 0, 0],
#               [0, 0, 0, 0],
#               [0, 0, 1, 0],
#               [0, 0, 0, 0],
#               [2, 5, 0, 0],
#               [0, 0, 0, 0],
#               [0, 0, 0, 0]], dtype=np.float64)


A = np.array([[1, 0, 1],
              [2, 5**.5, 0],
              [0, 0, 1],
              [0, 0, 1]])
B, Qt, P = SVD.svd_phaseI(A)
print(B)


[[-2.23606798e+00  2.04939015e+00 -9.45175510e-17]
 [ 1.11022302e-16  8.39500986e-01 -5.07346289e-01]
 [ 0.00000000e+00 -2.67014343e-18  1.68458833e+00]
 [ 0.00000000e+00 -4.39009789e-18 -3.33066907e-16]]


In [62]:
print(Qt @ A @ P)

[[-2.23606798e+00  2.04939015e+00  9.45175510e-17]
 [-1.11022302e-16  8.39500986e-01 -5.07346289e-01]
 [ 5.55111512e-17 -2.00911092e-17  1.68458833e+00]
 [ 8.12941988e-18 -9.34729845e-18 -4.52965907e-16]]


Todo: We may not need to form the reflection matrix H explicitly during Golub-Kahan bidiagonalization.

## Phase-II-A

**Test SVD**

In [63]:
%precision 4

'%.4f'

In [67]:
U, S, Vt = SVD.svd(A)
print(U@np.diag(S)@Vt)
U, S, Vt

[[-0.1586 -1.139   0.8231]
 [-2.2198 -1.9124 -0.6443]
 [-0.1535 -0.1509  0.9766]
 [-0.1535 -0.1509  0.9766]]


(array([[-0.2644,  0.6058, -0.7504],
        [-0.9631, -0.2059,  0.1732],
        [-0.035 ,  0.5435,  0.451 ],
        [-0.035 ,  0.5435,  0.451 ]]),
 array([3.0902, 1.7648, 0.5798]),
 array([[ 0.7089,  0.6969,  0.1082],
        [ 0.11  , -0.2608,  0.9591],
        [-0.6967,  0.668 ,  0.2615]]))

In [70]:
U, S, Vt  = scipy.linalg.svd(A, full_matrices=False)
print(U@np.diag(S)@Vt)
U, S, Vt

[[ 1.0000e+00 -1.2494e-16  1.0000e+00]
 [ 2.0000e+00  2.2361e+00 -1.8200e-16]
 [-2.9816e-17 -1.3617e-16  1.0000e+00]
 [-2.3894e-17 -1.1752e-16  1.0000e+00]]


(array([[-0.2644,  0.6058,  0.7504],
        [-0.9631, -0.2059, -0.1732],
        [-0.035 ,  0.5435, -0.451 ],
        [-0.035 ,  0.5435, -0.451 ]]),
 array([3.0902, 1.7648, 0.5798]),
 array([[-0.7089, -0.6969, -0.1082],
        [ 0.11  , -0.2608,  0.9591],
        [ 0.6967, -0.668 , -0.2615]]))